# Notebook to loop over events generated in Pythia's c++ version


In [1]:
import numpy as np
import sys
import dimod
import matplotlib.pyplot as plt
import os
import re
import neal

from dwave.system import EmbeddingComposite, DWaveSampler
from dwave_qbsolv import QBSolv
from pyqubo import Spin, Array
from embedding_generator import generate_embedding

In [2]:
# Define variable here

# Observed big difference in obtained minimum when increasing the number of reads from 100 to 1000
num_reads = 1000
num_events = 1000
path = '/home/andrea/pythiaEvents/'

theta = np.pi/2.

sampler = EmbeddingComposite(DWaveSampler(solver='Advantage_system1.1'))
sa_sampler = neal.SimulatedAnnealingSampler()

In [5]:

with open('/home/andrea/pythiaEvents/DWaveResults.dat', 'w') as fout:
    for ev in range(0, 2): #num_events):
        print(ev)

        fname = path + 'Event_' + str(ev) + '.dat'
        f = open(fname)

        pat = re.compile(r"\s+")

        g = list(zip(*[ pat.sub(" ",x.strip()).split() for x in f]))

        px = np.array([ float(x) for x in g[0]])
        py = np.array([ float(x) for x in g[1]])
        pz = np.array([ float(x) for x in g[2]])
        e = np.array([ float(x) for x in g[3]])

        # Generate qubo

        n_part = len(px)
        #print(n_part)
        s = Array.create('s', shape=n_part, vartype='BINARY')

        # Create an array of zeroes for qubo matrix coefficients
        coeff = [[0] * n_part for _ in range(n_part)] 

        for i in range(0,n_part):
            for j in range(0,n_part):
                coeff[i][j] = (px[i]*px[j] +py[i]*py[j] + pz[i]*pz[j] - e[i]*e[j] * np.cos(theta))/(1 - np.cos(theta))

        # Construct Hamiltonian
        H = sum([ -1.0*coeff[i][j]*s[i]*s[j] for i in range(0,n_part) for j in range(0,n_part)])

        # Compile model using pyqubo

        model = H.compile()
        qubo, offset = model.to_qubo()

        # Submit problem to D-Wave
        #response = QBSolv().sample_qubo(qubo)#
        response = sampler.sample_qubo(qubo,num_reads=num_reads)

        fnameout = '/home/andrea/pythiaEvents/Event_%s_qsolutions.dat' %ev
        with open(fnameout, 'w') as f:
            for p in range(n_part):
                key = 's[%s]' %p
                f.write("%s\n" %(response.first.sample[key]))
    
    #print('Finished annealing run on Advantage 1.1')

        print(response.first.energy)

        #response_sa = sa_sampler.sample_qubo(qubo)

    #print('Finished simmulated annealing routine')

    #print(response_sa.first.energy)

        sum_mom = sum([np.sqrt(px[i]*px[i] + py[i]*py[i]+ pz[i]*pz[i])  for i in range(0,n_part)])
    
        print(sum_mom)
        thrust = 2.*np.sqrt(-1.*response.first.energy)/sum_mom
        print(thrust)

        #thrust_sa = 2.*np.sqrt(-1.*response_sa.first.energy)/sum_mom
        #fout.write("%s\n" %(thrust))
    
    #print(response.data_vectors['sample'][0])
    # Store solutions

    #qpu_solutions = []

    #for datum in response.data(['sample','energy']):
    #    print(datum.energy)
    #    qpu_solutions.append(datum.sample)


0
-2012.0070412910209
90.86611912138152
0.9872855120605756
1
-1955.3636079430344
90.22468231234463
0.9802083508748164


In [10]:
with open('/home/andrea/pythiaEvents/Event_0_qsolutions.dat', 'w') as f:
    for p in range(n_part):
        key = 's[%s]' %p
        f.write("%s\n" %(response.first.sample[key]))

In [17]:
p = 0
response.first.sample['s[7]']

0

In [ ]:
thrust = sum([ -1.0*coeff[i][j]*response.first.sample['s['+str(i)+']']*response.first.sample['s['+str(j)+']'] for i in range(0,n_part) for j in range(0,n_part)])
print(thrust)

In [24]:
sum_mom = sum([np.sqrt(px[i]*px[i] + py[i]*py[i]+ pz[i]*pz[i])  for i in range(0,n_part)])

In [25]:
print(sum_mom)

90.86611912138152


In [ ]:
-thrust/(2*sum_mom)